In [0]:
# Putanja u Unity Catalog-u (Volumes)
path = "/Volumes/workspace/default/podaci/Real_Estate_Sales_2001-2022_spark.csv"

spark_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(path)

display(spark_df.limit(5))

In [0]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Inicijalizacija
spark = SparkSession.builder.appName("RealEstateAnalysis").getOrCreate()

# Putanja u Workspace katalogu
path = "/Volumes/workspace/default/podaci/Real_Estate_Sales_2001-2022_spark.csv"

# 1. Učitavanje u spark dataframe
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(path)

# 2. Registracija za SQL
df.createOrReplaceTempView("properties")

# SQL upit: Analiza po gradovima (agregacija i filtriranje)
query1 = spark.sql("""
    SELECT Town, 
           COUNT(*) as Broj_Prodaja, 
           ROUND(AVG(`Assessed Value`), 2) as Prosecna_Procenjena_Vrednost,
           ROUND(AVG(`Sale Amount`), 2) as Prosecna_Prodajna_Cena
    FROM properties
    WHERE `Property Type` = 'Residential'
    GROUP BY Town
    HAVING Broj_Prodaja > 5
    ORDER BY Prosecna_Prodajna_Cena DESC
""")

display(query1)

In [0]:
# Upit 2: Prosečna prodajna cena po godinama za stambene objekte
query2 = spark.sql("""
    SELECT `List Year`, 
           COUNT(*) as Broj_Prodaja, 
           ROUND(AVG(`Sale Amount`), 2) as Prosecna_Cena,
           MAX(`Sale Amount`) as Maksimalna_Cena
    FROM properties
    WHERE `Property Type` = 'Residential'
    GROUP BY `List Year`
    ORDER BY `List Year` ASC
""")
display(query2)

In [0]:
# Upit 3: Koji tip stambenog objekta ima najbolji odnos procenjene i prodajne cene?
query3 = spark.sql("""
    SELECT `Residential Type`, 
           COUNT(*) as Ukupno,
           ROUND(AVG(`Sales Ratio`), 4) as Prosecan_Sales_Ratio
    FROM properties
    WHERE `Residential Type` IS NOT NULL
    GROUP BY `Residential Type`
    ORDER BY Prosecan_Sales_Ratio DESC
""")
display(query3)

In [0]:
# Upit 4: Analiza tržišne premije po tipovima objekata
query4 = spark.sql("""
    SELECT 
        `Property Type`, 
        COUNT(*) as Broj_Prodaja,
        ROUND(AVG(`Sale Amount`), 2) as Prosecna_Prodajna_Cena
    FROM properties
    WHERE `Assessed Value` > 0 AND `Sale Amount` > 0
    GROUP BY `Property Type`
    HAVING Broj_Prodaja > 5
    ORDER BY Prosecna_Prodajna_Cena DESC
""")
display(query4)